In [80]:
# most of the code is borrowed from: https://www.tensorflow.org/tutorials/sequences/text_generation

In [4]:
import pandas as pd
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [285]:
input = pd.read_csv("fnames_manual_u.csv")

In [286]:
text = input.sum()['name'].lower()

In [287]:
vocab = sorted(set(text + " "))

In [288]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [289]:
def text_to_array(text):
    return np.array([char2idx[c] for c in text.lower() + " "])

In [290]:
input_data = input.applymap(text_to_array)['name'].values

In [291]:
dataset = tf.data.Dataset.from_generator(lambda: input_data, 
                                         tf.int64)

In [292]:
for value in dataset.take(3):
    print(value)

tf.Tensor([11  9 13  0], shape=(4,), dtype=int64)
tf.Tensor([12  6  3  0], shape=(4,), dtype=int64)
tf.Tensor([ 3 12  1 18  0], shape=(5,), dtype=int64)


In [293]:
for item in dataset.take(3):
  print(repr(''.join(idx2char[item.numpy()])))

'ким '
'лев '
'влас '


In [294]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [295]:
train_set = dataset.map(split_input_target)

In [296]:
for x in train_set.take(2):
    print(x)

(<tf.Tensor: id=13303140, shape=(3,), dtype=int64, numpy=array([11,  9, 13])>, <tf.Tensor: id=13303141, shape=(3,), dtype=int64, numpy=array([ 9, 13,  0])>)
(<tf.Tensor: id=13303144, shape=(3,), dtype=int64, numpy=array([12,  6,  3])>, <tf.Tensor: id=13303145, shape=(3,), dtype=int64, numpy=array([6, 3, 0])>)


In [297]:
for input_example, target_example in train_set.take(1):
  print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'ким'
Target data: 'им '


In [298]:
BATCH_SIZE = 1
examples_per_epoch = input_data.size
steps_per_epoch = examples_per_epoch//BATCH_SIZE
BUFFER_SIZE = 10000
print(BATCH_SIZE, examples_per_epoch, steps_per_epoch, BUFFER_SIZE)

1 170 170 10000


In [299]:
batched_trainset = train_set.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [300]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 64

# Number of RNN units
rnn_units = 256

In [301]:
import functools
rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [302]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [303]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [304]:
for input_example_batch, target_example_batch in batched_trainset.take(3): 
  #print(input_example_batch)
  example_batch_predictions = model(input_example_batch)
  #print(example_batch_predictions)
  #print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
  #print(example_batch_predictions[0])
  sampled_indices = tf.random.multinomial(example_batch_predictions[0], num_samples=1)
  sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
  #print(sampled_indices)
  print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
  print()
  print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices])))


Input: 
 'евдоким'

Next Char Predictions: 
 'мбтбюлж'
Input: 
 'святополк'

Next Char Predictions: 
 'нгефьёхйа'
Input: 
 'вадим'

Next Char Predictions: 
 'ьнпун'


In [305]:
def loss(labels, logits):
  return tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)

In [306]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [310]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints_names'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [308]:
EPOCHS=10

In [309]:
history = model.fit(batched_trainset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/10
170/170 [==============================] - 17s 103ms/step - loss: 2.9994
Epoch 2/10
170/170 [==============================] - 18s 105ms/step - loss: 2.4070
Epoch 3/10
170/170 [==============================] - 15s 89ms/step - loss: 2.2720
Epoch 4/10
170/170 [==============================] - 15s 87ms/step - loss: 2.1818
Epoch 5/10
170/170 [==============================] - 15s 88ms/step - loss: 2.1139
Epoch 6/10
170/170 [==============================] - 20s 117ms/step - loss: 2.0252
Epoch 7/10
170/170 [==============================] - 19s 111ms/step - loss: 1.9769
Epoch 8/10
170/170 [==============================] - 18s 108ms/step - loss: 1.8893
Epoch 9/10
170/170 [==============================] - 21s 121ms/step - loss: 1.8152
Epoch 10/10
170/170 [==============================] - 18s 107ms/step - loss: 1.7374


In [237]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [238]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [311]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (1, None, 64)             1920      
_________________________________________________________________
gru_8 (GRU)                  (1, None, 256)            246528    
_________________________________________________________________
dense_8 (Dense)              (1, None, 30)             7710      
Total params: 256,158
Trainable params: 256,158
Non-trainable params: 0
_________________________________________________________________


In [312]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 16

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated)).split(' ')[0]


In [342]:
print(generate_text(model, start_string=u"гри"))

гривафентё
